In [1]:
import pandas as pd
import numpy as np

In [25]:
hosp = pd.read_csv('../selenium-extraction/resultado.csv', decimal=".")

In [26]:
hosp.columns

Index(['hotel', 'cidade', 'preço', 'checkin', 'checkout', 'dist_centro',
       'dist_praia', 'avaliacao', 'qtnd_avaliacoes', 'data_extracao'],
      dtype='object')

In [27]:
hosp.dtypes

hotel               object
cidade              object
preço                int64
checkin             object
checkout            object
dist_centro        float64
dist_praia         float64
avaliacao          float64
qtnd_avaliacoes    float64
data_extracao       object
dtype: object

# Definindo os tipos das colunas

In [28]:
hosp['checkin'] = pd.to_datetime(hosp['checkin'])
hosp['checkout'] = pd.to_datetime(hosp['checkout'])
hosp['cidade'] = hosp["cidade"].astype("category")
hosp['hotel'] = hosp["hotel"].astype("category")
hosp['data_extracao'] =hosp["data_extracao"].astype("category")

In [29]:
hosp["cidade"].cat.categories

Index(['Lisboa', 'Paris'], dtype='object')

In [30]:
hosp["hotel"].cat.categories

Index(['#Private room C', '#Tram28_Cosy',
       '* Cozy & Renovated Studio in Paris *',
       '- SweetHome Lisbon sinta-se em casa -',
       '0#Studio 2pers#Champs Elysées#Arc de Triomphe',
       '0.Appart 4pers#Jardin Luxembourg#NECKER',
       '024 A - In the Heart of Lisbon',
       '024 B - Cosy Retreat in Madragoa -Lisbon',
       '024 C - Lisbon Tiles House', '024 D - Cosy Retreat in Lisbon',
       ...
       'xxxx xxxx', '´Sao Vicente III´- Eco-studio french balcony&view',
       'Áurea Museum by Eurostars Hotel Company', 'Échappée sur les toits',
       'Élégance Parisienne au cœur du Marais',
       'Élégant Studio Lumineux Paris',
       'Étage d’un duplex au 17ème arr de Paris  vue sur le Défense à 10min des champs Élysée',
       'ღ Le Jardin Parisien Calme avec Terrasse et Wifi ღ', '休闲民宿', '理想方向'],
      dtype='object', length=6677)

# Removendo Colunas

In [32]:
hosp.shape

(100650, 10)

In [31]:
print(hosp.isnull().sum())

hotel                   0
cidade                  0
preço                   0
checkin                 0
checkout                0
dist_centro             0
dist_praia         100650
avaliacao            3894
qtnd_avaliacoes      3894
data_extracao           0
dtype: int64


Observamos que a coluna "dist_praia" apresenta mais de 100.000 linhas sem valores. Desta forma fica viável a exclusão da coluna, visto que ela apresenta 100% das suas linhas vazias.